In [ ]:
import numpy as np
import pandas as pd

from neuralforecast.losses.numpy import mae, rmse

In [ ]:
def _evaluate_ohiot1dm(Y_hat_df, av_mask, model_name, metric, values):
    results_df = Y_hat_df.copy()
    # Filter values with at least 1 available mask in input window
    results_df = results_df.merge(av_mask[['unique_id', 'cutoff', 'sum_av_mask']], on=['unique_id', 'cutoff'], how='left')
    results_df = results_df[results_df['sum_av_mask'] > 0].reset_index(drop=True)
    # Filter ffill values of y
    results_df = results_df[results_df.available_mask==1]
    # Keep critical values of y

    if values == 'critical':
        results_df = results_df[(results_df.y<=70) | (results_df.y>=180)]

    if metric == 'mae':
        metric_result = mae(results_df['y'], results_df[model_name])
    elif metric == 'rmse':
        metric_result = rmse(results_df['y'], results_df[model_name])
    
    return metric_result

def _evaluate_simglucose(Y_hat_df, model_name, metric, values):
    results_df = Y_hat_df.copy()

    if values == 'critical':
        results_df = results_df[(results_df.y<=70) | (results_df.y>=180)]

    if metric == 'mae':
        metric_result = mae(results_df['y'], results_df[model_name])
    elif metric == 'rmse':
        metric_result = rmse(results_df['y'], results_df[model_name])
    
    return metric_result

def get_ohiot1dm_results(dataset, trials, model_name, approach, metric, values):
    metric_group = []
    
    for trial in trials:
        if 'exog' in dataset:
            base_df_name = dataset.replace('_exog', '')
        else:
            base_df_name = dataset
            
        results = pd.read_csv(f'./results/{dataset}_6/{approach}_models/trial_{approach}_{trial}/forecasts.csv')
        avmask = pd.read_csv(f'./data/{base_df_name}_exog_9_day_test_avmask.csv')
        metric_result = _evaluate_ohiot1dm(results, avmask, model_name, metric, values)
        metric_group.append(metric_result)
    
    return np.mean(np.array(metric_group)), np.std(np.array(metric_group))


def get_simglucose_results(dataset, trials, model_name, approach, metric, values):
    metric_group = []
    
    for trial in trials:
        results = pd.read_csv(f'./results/{dataset}_6/{approach}_models/trial_{approach}_{trial}/forecasts.csv')
        metric_result = _evaluate_simglucose(results, model_name, metric, values)
        metric_group.append(metric_result)
    
    return np.mean(np.array(metric_group)), np.std(np.array(metric_group))


## OhioT1DM - Baseline results

In [ ]:
model_names = ['AutoTFT']
metric = 'mae'
trials = range(8)
dataset = 'ohiot1dm'
approach = 'baseline'

mean_results_df1 = pd.DataFrame()
sd_results_df1 = pd.DataFrame()
for model_name in model_names:
    mm, ms = get_ohiot1dm_results(dataset, trials, model_name, approach, metric, values='all')
    final_mean = pd.DataFrame([[mm]], index=[model_name]).round(3)
    final_sd = pd.DataFrame([[ms]], index=[model_name]).round(3)
    mean_results_df1 = pd.concat([mean_results_df1, final_mean], axis=0)
    sd_results_df1 = pd.concat([sd_results_df1, final_sd], axis=0)

mean_results_df2 = pd.DataFrame()
sd_results_df2 = pd.DataFrame()
for model_name in model_names:
    mm, ms = get_ohiot1dm_results(dataset+'_exog', trials, model_name, approach, metric, values='all')
    final_mean = pd.DataFrame([[mm]], index=[model_name]).round(3)
    final_sd = pd.DataFrame([[ms]], index=[model_name]).round(3)
    mean_results_df2 = pd.concat([mean_results_df2, final_mean], axis=0)
    sd_results_df2 = pd.concat([sd_results_df2, final_sd], axis=0)

mean_results_df3 = pd.DataFrame()
sd_results_df3 = pd.DataFrame()
for model_name in model_names:
    mm, ms = get_ohiot1dm_results(dataset, trials, model_name, approach, metric, values='critical')
    final_mean = pd.DataFrame([[mm]], index=[model_name]).round(3)
    final_sd = pd.DataFrame([[ms]], index=[model_name]).round(3)
    mean_results_df3 = pd.concat([mean_results_df3, final_mean], axis=0)
    sd_results_df3 = pd.concat([sd_results_df3, final_sd], axis=0)

mean_results_df4 = pd.DataFrame()
sd_results_df4 = pd.DataFrame()
for model_name in model_names:
    mm, ms = get_ohiot1dm_results(dataset+'_exog', trials, model_name, approach, metric, values='critical')
    final_mean = pd.DataFrame([[mm]], index=[model_name]).round(3)
    final_sd = pd.DataFrame([[ms]], index=[model_name]).round(3)
    mean_results_df4 = pd.concat([mean_results_df4, final_mean], axis=0)
    sd_results_df4 = pd.concat([sd_results_df4, final_sd], axis=0)


print(pd.concat([mean_results_df1,
           mean_results_df2,
           mean_results_df3,
           mean_results_df4,
          ], axis=1)
     )
print(pd.concat([sd_results_df1,
           sd_results_df2,
           sd_results_df3,
           sd_results_df4,
          ], axis=1)
     )

## OhioT1DM - Treat Results

In [ ]:
model_names = ['AutoTFT']
metric = 'mae'
trials = range(8)
dataset = 'ohiot1dm'
approach = 'treat'

mean_results_df2 = pd.DataFrame()
sd_results_df2 = pd.DataFrame()
for model_name in model_names:
    mm, ms = get_ohiot1dm_results(dataset+'_exog', trials, model_name, approach, metric, values='all')
    final_mean = pd.DataFrame([[mm]], index=[model_name]).round(3)
    final_sd = pd.DataFrame([[ms]], index=[model_name]).round(3)
    mean_results_df2 = pd.concat([mean_results_df2, final_mean], axis=0)
    sd_results_df2 = pd.concat([sd_results_df2, final_sd], axis=0)

mean_results_df4 = pd.DataFrame()
sd_results_df4 = pd.DataFrame()
for model_name in model_names:
    mm, ms = get_ohiot1dm_results(dataset+'_exog', trials, model_name, approach, metric, values='critical')
    final_mean = pd.DataFrame([[mm]], index=[model_name]).round(3)
    final_sd = pd.DataFrame([[ms]], index=[model_name]).round(3)
    mean_results_df4 = pd.concat([mean_results_df4, final_mean], axis=0)
    sd_results_df4 = pd.concat([sd_results_df4, final_sd], axis=0)


print(pd.concat([mean_results_df2,
               mean_results_df4,
              ], axis=1)
     )
print(pd.concat([sd_results_df2,
               sd_results_df4,
              ], axis=1)
     )

## OhioT1DM - Sum Total Results

In [ ]:
model_names = ['AutoTFT']
metric = 'mae'
trials = range(8)
dataset = 'ohiot1dm'
approach = 'sum_total'

mean_results_df2 = pd.DataFrame()
sd_results_df2 = pd.DataFrame()
for model_name in model_names:
    mm, ms = get_ohiot1dm_results(dataset+'_exog', trials, model_name, approach, metric, values='all')
    final_mean = pd.DataFrame([[mm]], index=[model_name]).round(3)
    final_sd = pd.DataFrame([[ms]], index=[model_name]).round(3)
    mean_results_df2 = pd.concat([mean_results_df2, final_mean], axis=0)
    sd_results_df2 = pd.concat([sd_results_df2, final_sd], axis=0)

mean_results_df4 = pd.DataFrame()
sd_results_df4 = pd.DataFrame()
for model_name in model_names:
    mm, ms = get_ohiot1dm_results(dataset+'_exog', trials, model_name, approach, metric, values='critical')
    final_mean = pd.DataFrame([[mm]], index=[model_name]).round(3)
    final_sd = pd.DataFrame([[ms]], index=[model_name]).round(3)
    mean_results_df4 = pd.concat([mean_results_df4, final_mean], axis=0)
    sd_results_df4 = pd.concat([sd_results_df4, final_sd], axis=0)

print(pd.concat([mean_results_df2,
           mean_results_df4,
          ], axis=1)
     )
print(pd.concat([sd_results_df2,
           sd_results_df4,
          ], axis=1)
     )

## Simglucose - Baseline Results

In [ ]:
model_names = ['AutoTFT']
metric = 'rmse'
trials = range(8)
dataset = 'simglucose'
approach = 'baseline'

mean_results_df1 = pd.DataFrame()
sd_results_df1 = pd.DataFrame()
for model_name in model_names:
    mm, ms = get_simglucose_results(dataset, trials, model_name, approach, metric, values='all')
    final_mean = pd.DataFrame([[mm]], index=[model_name]).round(3)
    final_sd = pd.DataFrame([[ms]], index=[model_name]).round(3)
    mean_results_df1 = pd.concat([mean_results_df1, final_mean], axis=0)
    sd_results_df1 = pd.concat([sd_results_df1, final_sd], axis=0)

mean_results_df2 = pd.DataFrame()
sd_results_df2 = pd.DataFrame()
for model_name in model_names:
    mm, ms = get_simglucose_results(dataset+'_exog', trials, model_name, approach, metric, values='all')
    final_mean = pd.DataFrame([[mm]], index=[model_name]).round(3)
    final_sd = pd.DataFrame([[ms]], index=[model_name]).round(3)
    mean_results_df2 = pd.concat([mean_results_df2, final_mean], axis=0)
    sd_results_df2 = pd.concat([sd_results_df2, final_sd], axis=0)

mean_results_df3 = pd.DataFrame()
sd_results_df3 = pd.DataFrame()
for model_name in model_names:
    mm, ms = get_simglucose_results(dataset, trials, model_name, approach, metric, values='critical')
    final_mean = pd.DataFrame([[mm]], index=[model_name]).round(3)
    final_sd = pd.DataFrame([[ms]], index=[model_name]).round(3)
    mean_results_df3 = pd.concat([mean_results_df3, final_mean], axis=0)
    sd_results_df3 = pd.concat([sd_results_df3, final_sd], axis=0)

mean_results_df4 = pd.DataFrame()
sd_results_df4 = pd.DataFrame()
for model_name in model_names:
    mm, ms = get_simglucose_results(dataset+'_exog', trials, model_name, approach, metric, values='critical')
    final_mean = pd.DataFrame([[mm]], index=[model_name]).round(3)
    final_sd = pd.DataFrame([[ms]], index=[model_name]).round(3)
    mean_results_df4 = pd.concat([mean_results_df4, final_mean], axis=0)
    sd_results_df4 = pd.concat([sd_results_df4, final_sd], axis=0)


print(pd.concat([mean_results_df1,
           mean_results_df2,
           mean_results_df3,
           mean_results_df4,
          ], axis=1)
     )
print(pd.concat([sd_results_df1,
           sd_results_df2,
           sd_results_df3,
           sd_results_df4,
          ], axis=1)
     )

## Simglucose - Treat Results

In [ ]:
model_names = ['AutoTFT']
metric = 'mae'
trials = range(8)
dataset = 'simglucose'
approach = 'treat'

mean_results_df2 = pd.DataFrame()
sd_results_df2 = pd.DataFrame()
for model_name in model_names:
    mm, ms = get_simglucose_results(dataset+'_exog', trials, model_name, approach, metric, values='all')
    final_mean = pd.DataFrame([[mm]], index=[model_name]).round(3)
    final_sd = pd.DataFrame([[ms]], index=[model_name]).round(3)
    mean_results_df2 = pd.concat([mean_results_df2, final_mean], axis=0)
    sd_results_df2 = pd.concat([sd_results_df2, final_sd], axis=0)

mean_results_df3 = pd.DataFrame()
sd_results_df3 = pd.DataFrame()
for model_name in model_names:
    mm, ms = get_simglucose_results(dataset+'_exog', trials, model_name, approach, metric, values='critical')
    final_mean = pd.DataFrame([[mm]], index=[model_name]).round(3)
    final_sd = pd.DataFrame([[ms]], index=[model_name]).round(3)
    mean_results_df3 = pd.concat([mean_results_df3, final_mean], axis=0)
    sd_results_df3 = pd.concat([sd_results_df3, final_sd], axis=0)

print(pd.concat([mean_results_df2,
           mean_results_df3,
          ], axis=1)
     )
print(pd.concat([sd_results_df2,
           sd_results_df3,
          ], axis=1)
     )

## Simglucose - Sum Total Results

In [ ]:
model_names = ['AutoTFT']
metric = 'mae'
trials = range(8)
dataset = 'simglucose'
approach = 'sum_total'

mean_results_df2 = pd.DataFrame()
sd_results_df2 = pd.DataFrame()
for model_name in model_names:
    mm, ms = get_simglucose_results(dataset+'_exog', trials, model_name, approach, metric, values='all')
    final_mean = pd.DataFrame([[mm]], index=[model_name]).round(3)
    final_sd = pd.DataFrame([[ms]], index=[model_name]).round(3)
    mean_results_df2 = pd.concat([mean_results_df2, final_mean], axis=0)
    sd_results_df2 = pd.concat([sd_results_df2, final_sd], axis=0)

mean_results_df4 = pd.DataFrame()
sd_results_df4 = pd.DataFrame()
for model_name in model_names:
    mm, ms = get_simglucose_results(dataset+'_exog', trials, model_name, approach, metric, values='critical')
    final_mean = pd.DataFrame([[mm]], index=[model_name]).round(3)
    final_sd = pd.DataFrame([[ms]], index=[model_name]).round(3)
    mean_results_df4 = pd.concat([mean_results_df4, final_mean], axis=0)
    sd_results_df4 = pd.concat([sd_results_df4, final_sd], axis=0)

print(pd.concat([mean_results_df2,
          mean_results_df4,
          ], axis=1)
     )
print(pd.concat([sd_results_df2,
          sd_results_df4,
          ], axis=1)
     )